<a href="https://colab.research.google.com/github/iarondon3/End-to-End-Retail-Data-Ecosystem/blob/main/04-ETL-Data-Warehousing/etl_warehouse_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚙️ Step 1: Install ETL & OLAP Tools (DuckDB)

In [ ]:
import os
import time

start_time = time.time()
print("📦 Installing DuckDB, Faker & Analytics Libraries...")

# DuckDB is an in-process SQL OLAP database. Perfect for Colab.
# CORRECTION: Added 'faker' to the installation list
os.system("pip install duckdb pandas plotly faker > /dev/null 2>&1")

import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from faker import Faker
import random
from datetime import datetime, timedelta

elapsed = round(time.time() - start_time, 2)
print(f"✅ Setup Completed in {elapsed}s. Ready to build the Data Warehouse!")

> **🔍 What just happened?**

> We set up a **Modern Data Stack** environment within Colab.
> * **DuckDB:** Instead of a heavy server, we use DuckDB, an in-process SQL OLAP database that allows us to run high-performance analytical queries directly on local data.
> * **Libraries:** We installed `Plotly` to simulate the interactive dashboards usually found in BI tools like Metabase or Power BI.

# 🏗️ Step 2: Extract - Generate Raw Operational Data (OLTP)

We simulate a "messy" source system to demonstrate the need for cleaning.

Enter the volume of data to reconstruct the environment and click play

In [ ]:
# @markdown **Data Volume:**
TRANSACTIONS = 50000 # @param {type:"slider", min:5000, max:50000, step:5000}

fake = Faker()
print(f"🎲 Generating {TRANSACTIONS} raw transactions...")

# --- A. WALGREENS CATALOG ---
CATEGORIES_LIST = ["Prescription Drugs (Rx)", "Over-the-Counter (OTC)", "Pain Relief", "Vitamins & Supplements", "Skin Care", "Personal Care", "Household", "Snacks", "Beverages"]

PRODUCT_TEMPLATES = {
    "Prescription Drugs (Rx)": ["Metformin 500mg", "Lisinopril 10mg", "Atorvastatin 20mg"],
    "Over-the-Counter (OTC)": ["Allegra 24hr", "Zyrtec 10mg", "Claritin 24hr"],
    "Pain Relief": ["Ibuprofen 200mg", "Advil Liqui-Gels", "Tylenol Extra Strength"],
    "Vitamins & Supplements": ["Vitamin C 1000mg", "Vitamin D3 2000IU", "Omega-3 Fish Oil"],
    "Skin Care": ["CeraVe Moisturizing Cream", "Neutrogena Hydro Boost", "Vaseline Jelly"],
    "Personal Care": ["Head & Shoulders Shampoo", "Dove Deodorant", "Crest 3D White"],
    "Household": ["Clorox Wipes", "Tide Detergent", "Charmin Ultra Soft"],
    "Snacks": ["Lay's Classic Chips", "Doritos Nacho Cheese", "Oreo Cookies"],
    "Beverages": ["Coca-Cola 2L", "Pepsi 2L", "Gatorade Fruit Punch"]
}

# Generate Master Products
product_master = []
prod_id_counter = 1
for cat in CATEGORIES_LIST:
    items = PRODUCT_TEMPLATES.get(cat, ["Generic Item"])
    for name in items:
        base_cost = round(random.uniform(2.0, 50.0), 2)
        product_master.append({"id": prod_id_counter, "name": name, "category": cat, "unit_cost": base_cost})
        prod_id_counter += 1

# --- B. BRANCHES (Fixed: 1 Branch per State = 10 Branches) ---
STATES_LIST = ['FL', 'TX', 'CA', 'IL', 'NY', 'PA', 'NC', 'GA', 'OH', 'MI']
branch_master = []

# Create exactly 10 branches, mapping ID 1 -> FL, ID 2 -> TX, etc.
for i, state in enumerate(STATES_LIST, 1):
    branch_master.append({
        "id": i,
        "state": state
    })

# --- C. GENERATE TRANSACTIONS ---
raw_data = []

for _ in range(TRANSACTIONS):
    prod = random.choice(product_master)
    branch = random.choice(branch_master) # Pick one of the 10 stores

    markup = random.uniform(1.2, 1.6)
    sell_price = round(prod['unit_cost'] * markup, 2)

    raw_data.append({
        'transaction_id': fake.uuid4(),
        'date_str': fake.date_between(start_date='-1y', end_date='today').strftime("%Y-%m-%d"),
        'product_id': prod['id'],
        'product_name': prod['name'],
        'category': prod['category'],
        'branch_id': branch['id'],      # IDs will be 1 to 10
        'branch_state': branch['state'],
        'qty': random.randint(1, 4),
        'unit_price': sell_price,
        'unit_cost': prod['unit_cost']
    })

df_staging = pd.DataFrame(raw_data)
print(f"✅ Extraction Complete. Staging Data Shape: {df_staging.shape}")
print(f"   Unique Branches Generated: {df_staging['branch_id'].nunique()} (Should be 10)")
df_staging.head(3)

> **🔍 What just happened?**

> We simulated the extraction from a **Legacy OLTP System**.
> * **Staging Area:** We generated raw, unpolished transactional data. Notice that the data is "dirty" (dates are strings, costs are hidden) and flat.
> * **The Challenge:** In this raw format, answering questions like *"Which state is most profitable?"* is computationally expensive. This justifies the need for the ETL process.

#  🔄 Step 3: Transform - Business Logic & Star Schema Design

Modeling the Star Schema: Fact_Sales + Dimensions (Product, Customer, Branch, Date)

In [ ]:
print("🛠️ Starting ETL Transformation Pipeline...")

# --- 1. DATA ENRICHMENT ---

# A. Generate Customer Data
num_rows = len(df_staging)
# Sequential IDs for Customers
df_staging['customer_id'] = [random.randint(1, 1000) for _ in range(num_rows)]
df_staging['customer_first'] = [fake.first_name() for _ in range(num_rows)]
df_staging['customer_last'] = [fake.last_name() for _ in range(num_rows)]

# B. Calculate Financial Metrics
df_staging['date'] = pd.to_datetime(df_staging['date_str'])
df_staging['total_revenue'] = df_staging['qty'] * df_staging['unit_price']
df_staging['total_cost'] = df_staging['qty'] * df_staging['unit_cost']
df_staging['gross_margin'] = df_staging['total_revenue'] - df_staging['total_cost']
df_staging['margin_percentage'] = round((df_staging['gross_margin'] / df_staging['total_revenue']) * 100, 1)

# --- 2. DIMENSIONAL MODELING ---

# 🌟 DIMENSION 1: PRODUCT
dim_product = df_staging[['product_id', 'product_name', 'category']].drop_duplicates(subset=['product_id'])
dim_product = dim_product.rename(columns={'product_name': 'name', 'category': 'category_name'})
dim_product = dim_product.sort_values(by='product_id').reset_index(drop=True)

# 🌟 DIMENSION 2: CUSTOMER
dim_customer = df_staging[['customer_id', 'customer_first', 'customer_last']].drop_duplicates(subset=['customer_id'])
dim_customer['full_name'] = dim_customer['customer_first'] + " " + dim_customer['customer_last']
dim_customer = dim_customer[['customer_id', 'full_name']]
dim_customer = dim_customer.sort_values(by='customer_id').reset_index(drop=True)

# 🌟 DIMENSION 3: BRANCH
dim_branch = df_staging[['branch_id', 'branch_state']].drop_duplicates(subset=['branch_id'])
dim_branch = dim_branch.rename(columns={'branch_state': 'state'})
# FORCE SORT to ensure 1, 2, 3... order in the final table
dim_branch = dim_branch.sort_values(by='branch_id').reset_index(drop=True)

# 🌟 DIMENSION 4: DATE (SURROGATE KEY)
unique_dates = df_staging[['date']].drop_duplicates().sort_values(by='date').reset_index(drop=True)
unique_dates['date_id'] = range(1, len(unique_dates) + 1)
unique_dates['year'] = unique_dates['date'].dt.year
unique_dates['quarter'] = unique_dates['date'].dt.quarter
unique_dates['month'] = unique_dates['date'].dt.month
unique_dates['month_name'] = unique_dates['date'].dt.month_name()
unique_dates['day_name'] = unique_dates['date'].dt.day_name()
dim_date = unique_dates[['date_id', 'date', 'year', 'quarter', 'month', 'month_name', 'day_name']]

# 🌟 FACT TABLE PREP (MERGE)
df_fact_merged = pd.merge(df_staging, unique_dates[['date', 'date_id']], on='date', how='left')

# 🌟 FACT TABLE: SALES
fact_sales = df_fact_merged[[
    'transaction_id',
    'date_id',
    'product_id',
    'customer_id',
    'branch_id',
    'qty',
    'unit_price',
    'unit_cost',
    'total_revenue',
    'gross_margin',
    'margin_percentage'
]]

print("✅ Transformation Complete! Star Schema Constructed.")

# --- 3. FINAL SCHEMA AUDIT ---
print("\n🔎 FINAL SCHEMA AUDIT (Headers + 3 Rows Sample):")
print("="*80)

dataframes = {
    "📦 Dim_Product": dim_product,
    "📍 Dim_Branch": dim_branch,
    "👥 Dim_Customer": dim_customer,
    "📅 Dim_Date": dim_date,
    "💰 Fact_Sales": fact_sales
}

for name, df in dataframes.items():
    print(f"\n{name}  [Shape: {df.shape}]")
    print("-" * 50)
    # Showing 3 rows to confirm sequence 1, 2, 3
    print(df.head(3).to_string(index=False))

> **🔍 What just happened?**

> We executed the **Transformation Engine** (Python/Pandas).
> * **Dimensional Modeling:** We decomposed the flat data into a **Star Schema**, creating distinct Dimensions (`Product`, `Branch`, `Customer`, `Date`) and a central Fact Table `Sales`.
> * **Business Logic:** We applied financial rules to calculate **Gross Margin** and **Profitability %**, enriching the data before it hits the warehouse.
> * **Data Quality:** We enforced sorted Surrogate Keys (IDs 1, 2, 3...) to ensure clean and indexable relationships.

# 💾 Step 4: Load - Building the Star Schema in DuckDB

 Loading the transformed DataFrames into the OLAP Engine.

In [ ]:
print("🏗️ Initializing Data Warehouse (DuckDB)...")
# We connect to an in-memory database.
# In a real scenario, this would be a connection string to AWS Redshift or Snowflake.
con = duckdb.connect(database=':memory:')

# --- THE LOAD PROCESS (Python -> SQL) ---
# This is where the DataFrames become real SQL Tables
print("   Loading Dimensions...")
con.execute("CREATE TABLE Dim_Product AS SELECT * FROM dim_product")
con.execute("CREATE TABLE Dim_Customer AS SELECT * FROM dim_customer")
con.execute("CREATE TABLE Dim_Branch AS SELECT * FROM dim_branch")
con.execute("CREATE TABLE Dim_Date AS SELECT * FROM dim_date")

print("   Loading Fact Table...")
con.execute("CREATE TABLE Fact_Sales AS SELECT * FROM fact_sales")

# --- POST-LOAD VERIFICATION ---
print("\n✅ Load Complete. The Star Schema is ready inside DuckDB!")

print("\n🔎 DATABASE CHECK (Running SQL 'SHOW TABLES'):")
# This query proves the tables exist in the DB engine
tables = con.execute("SHOW TABLES").df()
print(tables.to_string(index=False))

print("\n🔎 INTEGRITY CHECK (Testing a SQL Join):")
# Let's prove the tables are connected. We join Fact to Branch to see if it works.
sql_test = """
    SELECT
        b.state,
        COUNT(*) as total_sales,
        SUM(f.total_revenue) as revenue
    FROM Fact_Sales f
    JOIN Dim_Branch b ON f.branch_id = b.branch_id
    GROUP BY b.state
    ORDER BY revenue DESC
    LIMIT 3
"""
print("   Executing analytical query to verify relationships...")
print(con.execute(sql_test).df().to_string(index=False))

> **🔍 What just happened?**

> We performed the **Load (L)** phase, persisting data into the Data Warehouse.
> * **From Memory to SQL:** We moved the Python DataFrames into **DuckDB Tables**. Now the data is no longer just variables in RAM, but structured SQL tables.
> * **Integrity Check:** The final SQL query proved that our Star Schema works: we successfully joined `Fact_Sales` with `Dim_Branch` to aggregate revenue by State, confirming the relationships are valid.

# 📊 Step 5: Automated BI Reporting (Data Storytelling)


Strategy: Instead of manual widgets, we automate the generation of insights.

 1. Generate National Report.
 2. Detect Top Performing State via SQL.
 3. Generate Regional Deep-Dive automatically.

In [ ]:
import plotly.express as px
import plotly.io as pio

# Force Colab Rendering
pio.renderers.default = 'colab'

def generate_dashboard(filter_value):
    print(f"\n{'='*60}")
    if filter_value == 'All USA':
        print(f"🌎 GENERATING REPORT: NATIONAL OVERVIEW")
        where_clause = ""
        context_sql = """
            SELECT b.state, SUM(f.total_revenue) as Metric
            FROM Fact_Sales f
            JOIN Dim_Branch b ON f.branch_id = b.branch_id
            GROUP BY b.state ORDER BY Metric DESC
        """
        chart_title = "📍 Revenue by State"
        col_name = 'state'
        color_scale = 'Blues'
    else:
        print(f"⭐ GENERATING DEEP-DIVE: {filter_value}")
        where_clause = f"WHERE b.state = '{filter_value}'"
        context_sql = f"""
            SELECT p.name as Product, SUM(f.total_revenue) as Metric
            FROM Fact_Sales f
            JOIN Dim_Product p ON f.product_id = p.product_id
            JOIN Dim_Branch b ON f.branch_id = b.branch_id
            {where_clause}
            GROUP BY p.name ORDER BY Metric DESC LIMIT 5
        """
        chart_title = f"🏆 Top 5 Best Sellers in {filter_value}"
        col_name = 'Product'
        color_scale = 'Oranges'
    print(f"{'='*60}\n")

    # --- EXECUTE QUERIES ---
    # 1. KPI Category
    df_cat = con.execute(f"""
        SELECT p.category_name, SUM(f.total_revenue) as Revenue, SUM(f.gross_margin) as Margin
        FROM Fact_Sales f
        JOIN Dim_Product p ON f.product_id = p.product_id
        JOIN Dim_Branch b ON f.branch_id = b.branch_id
        {where_clause}
        GROUP BY p.category_name ORDER BY Revenue DESC
    """).df()

    # 2. KPI Trend
    df_trend = con.execute(f"""
        SELECT d.year, d.month, d.month_name, SUM(f.total_revenue) as Revenue, SUM(f.gross_margin) as Margin
        FROM Fact_Sales f
        JOIN Dim_Date d ON f.date_id = d.date_id
        JOIN Dim_Branch b ON f.branch_id = b.branch_id
        {where_clause}
        GROUP BY d.year, d.month, d.month_name ORDER BY d.year, d.month
    """).df()
    df_trend['Period'] = df_trend['month_name'] + " " + df_trend['year'].astype(str)

    # 3. Context Data
    df_geo = con.execute(context_sql).df()

    # --- RENDER CHARTS ---
    fig1 = px.bar(
        df_cat, x='category_name', y=['Revenue', 'Margin'],
        title=f"💰 Profitability by Category ({filter_value})",
        barmode='group', height=300,
        color_discrete_map={'Revenue': '#336791', 'Margin': '#47A248'}
    )
    fig1.show()

    fig2 = px.line(
        df_trend, x='Period', y=['Revenue', 'Margin'],
        title=f"📈 Sales Trend ({filter_value})",
        markers=True, height=300
    )
    fig2.show()

    fig3 = px.bar(
        df_geo, x=col_name, y='Metric',
        title=chart_title, text_auto='.2s', height=300,
        color='Metric', color_continuous_scale=color_scale
    )
    fig3.show()

# --- RUN AUTOMATED STORY ---

# 1. Show National Data
generate_dashboard('All USA')

# 2. Find the "Best" State automatically via SQL
print("\n🤖 AI ANALYST: Detecting Top Performing State...")
top_state_query = """
    SELECT b.state
    FROM Fact_Sales f
    JOIN Dim_Branch b ON f.branch_id = b.branch_id
    GROUP BY b.state
    ORDER BY SUM(f.total_revenue) DESC
    LIMIT 1
"""
best_state = con.execute(top_state_query).fetchone()[0]
print(f"✅ Detection Complete. Top Market is: {best_state}")

# 3. Show Top State Data
generate_dashboard(best_state)

> **🔍 What just happened?**

> We executed an **Automated Data Storytelling** script.
> * **Batch Reporting:** Instead of waiting for user clicks, the system automatically generated a **National Overview**.
> * **SQL Logic Injection:** The script then queried the Data Warehouse to identify the **Top Performing State** dynamically (detecting the winner based on current data).
> * **Contextual Deep-Dive:** Upon detection, it automatically re-ran the dashboard filters to generate a specific report for that state, showing its top-selling products. This simulates a nightly automated email report sent to regional managers.